In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")
gender_submission = pd.read_csv("../input/titanic/gender_submission.csv")

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [7]:
gender_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [8]:
data = pd.concat([train, test], sort=False)

In [9]:
data.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [10]:
data["Sex"].replace(["male", "female"], [0, 1], inplace=True)

In [11]:
data["Embarked"].replace(["S", "C", "Q"], [0, 1, 2], inplace=True)
data["Embarked"].fillna(data.Embarked.median(), inplace=True)

In [12]:
data["Fare"].fillna(data.Fare.median(), inplace=True)

In [13]:
data["Age"].fillna(data.Age.median(), inplace=True)

In [14]:
delete_columns = ["Name", "PassengerId",  "Ticket", "Cabin"]
data.drop(delete_columns, axis=1, inplace=True)

In [16]:
train = data[:len(train)]
test = data[len(train):]

In [17]:
Y_train = train["Survived"]
X_train = train.drop("Survived", axis=1)
X_test = test.drop("Survived", axis=1)

In [18]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [19]:
# hold out
train_x, valid_x, train_y, valid_y = train_test_split(X_train, Y_train, test_size=0.3, random_state=0, stratify=Y_train)

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]
    
    lgb_train = lgb.Dataset(X_tr, y_tr,
                                             categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train,
                                            categorical_feature=categorical_features)

    model = lgb.train(params, lgb_train,
                                   valid_sets=[lgb_train, lgb_eval],
                                   verbose_eval=10,
                                   num_boost_round=1000,
                                   early_stopping_rounds=10)

    oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    y_preds.append(y_pred)
    models.append(model)

In [20]:
categorical_features = ["Sex", "Embarked", "Pclass"]

In [1]:
import optuna
from sklearn.metrics import log_loss

In [22]:
def objectives(trial):
    params = {
        "objective": "binary",
        "max_bin": trial.suggest_int("maxi_bin", 255, 500)
        "learning_rate": 0.05,
        "num_leaves": trial.suggest_int("num_leaves", 32, 128)
    }
    
    lgb_train = lgb.Dataset(X_train, Y_train, categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(valid_x, valid_y, reference=lgb_train, categorical_feature=categorical_features)

    model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_eval], verbose_eval=10, num_boost_round=1000, early_stopping_rounds=10)
    
    y_pred_valid = model.predict(valid_x, num_iteration=model.best_iteration)
    
    score = log_loss(y_valid, y_pred_valid)
    return score

/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.420947	valid_1's binary_logloss: 0.423931
[20]	training's binary_logloss: 0.34163	valid_1's binary_logloss: 0.347112
[30]	training's binary_logloss: 0.302305	valid_1's binary_logloss: 0.30602
[40]	training's binary_logloss: 0.269003	valid_1's binary_logloss: 0.271767
[50]	training's binary_logloss: 0.244623	valid_1's binary_logloss: 0.248899
[60]	training's binary_logloss: 0.223279	valid_1's binary_logloss: 0.228391
[70]	training's binary_logloss: 0.2058	valid_1's binary_logloss: 0.212053
[80]	training's binary_logloss: 0.190524	valid_1's binary_logloss: 0.195664
[90]	training's binary_logloss: 0.178168	valid_1's binary_logloss: 0.183591
[100]	training's binary_logloss: 0.167367	valid_1's binary_logloss: 0.173395
[110]	training's binary_logloss: 0.158229	valid_1's binary_logloss: 0.164861
[120]	training's binary_logloss: 0.150219	valid_1's binary_logloss: 0.156826
[130]	training's binary_logl

In [ ]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)
study.best_params

In [24]:
y_pred

array([6.69426370e-04, 1.48420973e-07, 5.69997434e-02, 9.90847417e-01,
       1.56199478e-01, 1.93794524e-02, 2.52204273e-06, 4.73570681e-07,
       9.99733407e-01, 2.58667772e-07, 5.68896210e-08, 4.40208309e-04,
       9.99999293e-01, 5.01334373e-04, 9.88666972e-01, 9.99999984e-01,
       1.40742954e-04, 9.87708588e-01, 8.83732812e-01, 8.96561146e-04,
       1.17403605e-01, 1.18028621e-02, 9.99999975e-01, 9.81644176e-01,
       9.99807206e-01, 2.25647832e-06, 1.00000000e+00, 9.81787977e-01,
       9.95733997e-01, 1.88140935e-03, 3.79709742e-04, 1.98327839e-04,
       9.34150831e-01, 1.97330227e-04, 9.83623407e-01, 5.70784513e-01,
       3.47226710e-04, 3.33387757e-05, 4.61775937e-06, 5.01088137e-01,
       5.14448373e-05, 6.66721471e-01, 5.01750771e-01, 9.98938420e-01,
       9.99991157e-01, 6.92141489e-05, 6.59738369e-01, 1.04226036e-01,
       9.99968958e-01, 9.98938535e-01, 9.67106823e-01, 8.55502779e-04,
       9.99646669e-01, 9.01649874e-01, 2.09651924e-01, 1.49328466e-11,
      

In [25]:
pred = (y_pred > 0.5).astype(int)
pred

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [26]:
sub = pd.read_csv("../input/titanic/gender_submission.csv")
sub['Survived'] = pred
sub.to_csv('../output/submission.csv', index = False)